In [36]:
from flask import Flask, request, render_template
from binance.client import Client
from binance.enums import *
import pandas as pd
import json

In [37]:
import config
import binance_script as my_bnc

In [48]:
# receive webhook

wbhook = {
"passphrase":"mybot_12345",
"base_currency":"USDT",

"symbol":"BTCUSDT",
"side":"SELL",

"buy_fixed_or_ratio":"fixed",
"sell_fixed_or_ratio":"ratio",

"buy_ratio_percent":"100",
"sell_ratio_percent":"100",
"buy_fixed_amount_USDT":"20",
"sell_fixed_amount_USDT":"10",

"take_profit_percent":"30",
"stop_loss_percent":"20",

"trailing_stop_(no/historical/callback)":"no",
"trailing_stop_activation_percent":"20",
"trailing_stop_historical_bar":"10",
"trailing_stop_callback_rate":"20"
}

In [49]:
# get webhook wbhook

passphrase = wbhook['passphrase']
base_currency = wbhook['base_currency']

symbol = wbhook['symbol']
target_symbol = symbol.split(base_currency)[0]

side = wbhook['side']

buy_ratio_percent = float(wbhook['buy_ratio_percent'])
buy_fixed_amount_USDT = float(wbhook['buy_fixed_amount_USDT'])
buy_fixed_or_ratio = wbhook['buy_fixed_or_ratio']

sell_ratio_percent = float(wbhook['sell_ratio_percent'])
sell_fixed_amount_USDT = float(wbhook['sell_fixed_amount_USDT'])
sell_fixed_or_ratio = wbhook['sell_fixed_or_ratio']

take_profit_percent = float(wbhook['take_profit_percent'])
stop_loss_percent = float(wbhook['stop_loss_percent'])

trailing_stop_Type = wbhook['trailing_stop_(no/historical/callback)']
trailing_stop_activation_percent = float(wbhook['trailing_stop_activation_percent'])
trailing_stop_historical_bar = float(wbhook['trailing_stop_historical_bar'])
trailing_stop_callback_rate = float(wbhook['trailing_stop_callback_rate'])

#################################################################################################

print(f"passphrase:{passphrase}\n\nsymbol:{symbol}\ntarget_symbol:{target_symbol}\nbase_currency:{base_currency}\nside:{side}\n\nbuy_ratio_percent:{buy_ratio_percent}\nbuy_fixed_amount_USDT:{buy_fixed_amount_USDT}\nbuy_fixed_or_ratio:{buy_fixed_or_ratio}\n\nsell_ratio_percent:{sell_ratio_percent}\nsell_fixed_amount_USDT:{sell_fixed_amount_USDT}\nsell_fixed_or_ratio:{sell_fixed_or_ratio}\ntake_profit_percent:{take_profit_percent}\nstop_loss_percent:{stop_loss_percent}\n\ntrailing_stop_Type:{trailing_stop_Type}\ntrailing_stop_activation_percent:{trailing_stop_activation_percent}\ntrailing_stop_historical_bar:{trailing_stop_historical_bar}\ntrailing_stop_callback_rate:{trailing_stop_callback_rate}")


passphrase:mybot_12345

symbol:BTCUSDT
target_symbol:BTC
base_currency:USDT
side:SELL

buy_ratio_percent:100.0
buy_fixed_amount_USDT:20.0
buy_fixed_or_ratio:fixed

sell_ratio_percent:100.0
sell_fixed_amount_USDT:10.0
sell_fixed_or_ratio:ratio
take_profit_percent:30.0
stop_loss_percent:20.0

trailing_stop_Type:no
trailing_stop_activation_percent:20.0
trailing_stop_historical_bar:10.0
trailing_stop_callback_rate:20.0


In [50]:
# check user
if passphrase == config.WEBHOOK_PASSPHRASE_1:
    client = Client(config.API_KEY_1, config.API_SECRET_1)
    user_no = 1
elif passphrase == config.WEBHOOK_PASSPHRASE_2:
    client = Client(config.API_KEY_2, config.API_SECRET_2)
    user_no = 2
else:
    user_no = "N/A"

# check account balance
try:
    base_balance = float(my_bnc.check_balance(base_currency, user_no))
except Exception as e:
    print("an error occured - {}".format(e))

# check account balance (target symbol)
try:
    target_balance = float(my_bnc.check_balance(target_symbol, user_no))
except Exception as e:
    print("an error occured - {}".format(e))

USDT: 87.5830843
BTC: 0.00088


In [51]:
print(base_balance)
print(target_balance)

87.5830843
0.00088


In [52]:
# get all tick
all_price = client.get_all_tickers()
df_all_price = pd.DataFrame(all_price)

# check target price
price = df_all_price[(df_all_price.symbol == symbol)]
price = float(price['price'])
price

22587.35

In [53]:
# Position Sizing

# BUY

if buy_fixed_or_ratio == "fixed":
    buy_amount = buy_fixed_amount_USDT / price
    buy_amount

elif buy_fixed_or_ratio == "ratio":
    buy_amount = (base_balance * buy_ratio_percent) / price
    buy_amount

else:
    buy_amount = 0

# SELL

if sell_fixed_or_ratio == "fixed":
    sell_amount = sell_fixed_amount_USDT / price
    sell_amount

elif sell_fixed_or_ratio == "ratio":
    sell_amount = (target_balance * sell_ratio_percent) / 100
    sell_amount

else:
    sell_amount = 0

In [54]:
# order execution

if side == "BUY":
    order_amount = round(buy_amount,5)
elif side == "SELL":
    order_amount = round(sell_amount,5)
else:
    order_amount = "N/A"

print(buy_amount)
print(target_balance)
print(order_amount)

0.0008854513699039507
0.00088
0.00088


In [55]:
print(symbol)
print(side)
print(round(order_amount,5))

BTCUSDT
SELL
0.00088


In [56]:
order = client.create_order(symbol=symbol, side=side, type=ORDER_TYPE_MARKET, quantity=order_amount)


In [57]:
print(order)

{'symbol': 'BTCUSDT', 'orderId': 11774056617, 'orderListId': -1, 'clientOrderId': 'IPflBglP1XVbmizpTDcLyK', 'transactTime': 1658417992157, 'price': '0.00000000', 'origQty': '0.00088000', 'executedQty': '0.00088000', 'cummulativeQuoteQty': '19.87671840', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '22587.18000000', 'qty': '0.00088000', 'commission': '0.00000000', 'commissionAsset': 'BNB', 'tradeId': 1502965995}]}
